# Self Supervised Learning Basics

**Self-supervised learning (SSL)** is one of the most revolutionary ideas in modern machine learning. It allows models to learn **useful representations** of data without the need for manual labeling.

In SSL, the system creates its own supervision from the data : learning from **pseudo-labels** or **pretext tasks**.

---

## 🎯 Objectives
- Understand what self supervised learning is and why it matters.
- Learn about common SSL tasks in vision and NLP.
- Implement a simple example of contrastive learning using images.

---

## 🔍 What is Self-Supervised Learning?

In traditional **supervised learning**, models require labeled data, like:

| Input | Label |
|--------|--------|
| 🐶 Dog image | “Dog” |
| 🐱 Cat image | “Cat” |

But labeling data is expensive and time-consuming.

Self supervised learning solves this by using the **data itself as supervision**. The model generates *pseudo labels* automatically.

For example:
- Mask part of an image and ask the model to predict the missing region.
- Mask words in a sentence and predict the masked tokens (like in **BERT**).
- Compare different views (augmentations) of the same data point and make their embeddings similar (**Contrastive Learning**).

---

## 🧩 Common Self-Supervised Learning Tasks

| Domain | Task | Example |
|---------|------|----------|
| **Vision** | Image colorization | Predict colors from grayscale images |
| | Rotation prediction | Predict rotation angle (0°, 90°, 180°, 270°) |
| | Contrastive learning | SimCLR, MoCo, BYOL |
| **NLP** | Masked language modeling | Predict missing words (BERT) |
| | Next sentence prediction | Determine if one sentence follows another |
| | Autoencoding | Reconstruct input text |

---

## 🧠 Key Idea: Contrastive Learning

Contrastive learning teaches the model to bring **similar samples closer** and **push dissimilar ones apart** in the feature space.

### Example:
- Two augmented versions of the same image → **positive pair**
- Two images from different classes → **negative pair**

The model learns using a **contrastive loss function**, such as **InfoNCE loss**:

$$ L = -\log \frac{\exp(sim(z_i, z_j)/\tau)}{\sum_k \exp(sim(z_i, z_k)/\tau)} $$

where:
- \( sim(z_i, z_j) \): cosine similarity
- \( \tau \): temperature parameter

---

## 🧪 Example: Simple Contrastive Learning with Augmented Images

Let's see a minimal PyTorch implementation to illustrate the concept.


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision import datasets
from torch.utils.data import DataLoader

# Define simple augmentations
transform = T.Compose([
    T.RandomResizedCrop(32),
    T.RandomHorizontalFlip(),
    T.ToTensor()
])

# Load dataset
dataset = datasets.CIFAR10(root="./data", download=True, transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*32*3, 512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )
    def forward(self, x):
        return self.net(x)

# Cosine similarity loss (contrastive-like)
def contrastive_loss(z_i, z_j, temperature=0.5):
    sim = torch.nn.functional.cosine_similarity(z_i, z_j)
    return 1 - sim.mean()

encoder = Encoder()
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-3)

for imgs, _ in loader:
    # Two augmented views of same images
    aug1 = imgs + 0.05 * torch.randn_like(imgs)
    aug2 = imgs + 0.05 * torch.randn_like(imgs)
    z_i = encoder(aug1)
    z_j = encoder(aug2)
    loss = contrastive_loss(z_i, z_j)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    break

print(f"Contrastive loss: {loss.item():.4f}")

---
## 🔬 Real World Applications
- **Vision Transformers (ViT)** pre-trained with self supervised learning.
- **BERT**, **GPT**, and **SimCLR** are all based on self supervised principles.
- Used in **medical imaging**, **speech recognition**, and **recommendation systems**.

---

## ✅ Summary
- Self supervised learning allows models to **learn from unlabeled data**.
- Core idea: **create tasks where the input itself provides supervision**.
- Common techniques include **contrastive learning**, **masked prediction**, and **autoencoding**.

---

### 🚀 Next: Meta Learning Basics → `02-Meta_Learning_Basics.ipynb`
